In [6]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [50]:
def setDocumentContent():
    with open('/Users/sameernawaz/AnacondaProjects/Datasets/trainingTextClassification.csv', 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        count = 0
        for line in csv_reader:
            if len(line[5]) > sizeOfTweetContent:
                val[count] = line
                count += 1
                if count >= sizeOfSets :
                    break
            
    countTemp = 0
    for i in val:
        document[countTemp] = i[5]
        countTemp += 1

In [64]:
sizeOfSets = 10000
sizeOfTweetContent = 75
val = [None] * sizeOfSets
document = [""] * sizeOfSets

setDocumentContent()

In [65]:
!pip install textsearch
!pip install contractions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sameernawaz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sameernawaz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
print(val[0])

['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY', '_TheSpecialOne_', "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"]


In [67]:
df = pd.DataFrame(val, columns = ['Inf', 'ID', 'Time', 'Query', 'Username', 'Tweet']) 

In [68]:
df.head()

,Inf,ID,Time,Query,Username,Tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
3,0,1467812784,Mon Apr 06 22:20:20 PDT 2009,NO_QUERY,bayofwolves,"@smarrison i would've been the first, but i di..."
4,0,1467812799,Mon Apr 06 22:20:20 PDT 2009,NO_QUERY,HairByJess,@iamjazzyfizzle I wish I got to watch it with ...


In [69]:
df = df[['ID', 'Username', 'Tweet']]
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 3 columns):
ID          10000 non-null object
Username    10000 non-null object
Tweet       10000 non-null object
dtypes: object(3)
memory usage: 312.5+ KB


In [70]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['Tweet']))
len(norm_corpus)

10000

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(10000, 14490)

In [89]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,1.000000,0.0,0.0,0.0,0.043087,0.0,0.00000,0.0,0.0,0.030226,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
1,0.000000,1.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.027424,0.0,0.000000,0.0,0.000000,0.0
2,0.000000,0.0,1.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.097185,0.0,0.000000,0.0
3,0.000000,0.0,0.0,1.0,0.000000,0.0,0.04746,0.0,0.0,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.048391,0.0
4,0.043087,0.0,0.0,0.0,1.000000,0.0,0.00000,0.0,0.0,0.000000,...,0.0,0.134259,0.059122,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0


In [77]:
movies_list = df['ID'].values
movies_list, movies_list.shape

(array(['1467810369', '1467810672', '1467811193', ..., '1573084598',
        '1573084966', '1573085390'], dtype=object), (10000,))

In [80]:
movie_idx = np.where(movies_list == '1573085390')[0][0]
movie_idx

9999

In [81]:
movie_similarities = doc_sim_df.iloc[movie_idx].values
movie_similarities

array([0., 0., 0., ..., 0., 0., 1.])

In [82]:
similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
similar_movie_idxs

array([ 273, 5914, 9139, 5060, 1747])

In [83]:
similar_movies = movies_list[similar_movie_idxs]
similar_movies

array(['1468066505', '1557398038', '1564873910', '1556697259',
       '1469383992'], dtype=object)

In [84]:
def movie_recommender(movie_title, movies=movies_list, doc_sims=doc_sim_df):
    # find movie id
    movie_idx = np.where(movies == movie_title)[0][0]
    # get movie similarities
    movie_similarities = doc_sims.iloc[movie_idx].values
    # get top 5 similar movie IDs
    similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
    # get top 5 movies
    similar_movies = movies[similar_movie_idxs]
    # return the top 5 movies
    return similar_movies

In [87]:
print((df.loc[df['ID'] == "1573085390"]['Tweet']).values[0])
print('\n')
for i in movie_recommender(movie_title = "1573085390", movies=movies_list, doc_sims=doc_sim_df):
    print("=>",(df.loc[df['ID'] == i]['Tweet']).values[0])

@msSinCera THE LADY IN REDDDD IS DANCSHING WIF MEEEE ok random. but i need a break too  from the ulcer on my lidah that is


=> @octopuseatspie I GOT THE I CAN HAS CHEZBURGER BOOK FROM THE LOBO, AND YOU ARE NOT HERE TO LOOK AT IT WIF ME 
=> ARGHHHH...have misplaced my latest draft of the comedy show Ive been writing!!!!!! Think I need a break 
=> Back to my desk.  I think I'll nip outside when I need a break in a bit. It's too nice out there to be in here! Laptop would be ideal now!
=> Studying for math test on Tuesday. After Tuesday I can finally focus on my lab projects for CECS 346. I need a break. 
=> just had a mam as talk wif dad bout...shit. :@ still underhouse arrest. BASICALLY: ALL MY TRUST=GONE 
